In [ ]:
!git clone https://github.com/aka334/ConvLab-2.git
%cd ConvLab-2
!pip install -e .

Cloning into 'ConvLab-2'...
remote: Enumerating objects: 2621, done.
remote: Counting objects: 100% (338/338), done.
remote: Compressing objects: 100% (245/245), done.
remote: Total 2621 (delta 134), reused 213 (delta 92), pack-reused 2283
Receiving objects: 100% (2621/2621), 358.53 MiB | 13.45 MiB/s, done.
Resolving deltas: 100% (1328/1328), done.
Updating files: 100% (991/991), done.
/content/ConvLab-2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/ConvLab-2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 41.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 674.8/674.8 kB 46.2 MB/s eta 0:00:00
     ━━━━━━━

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.3 MB/s eta 0:00:00
  Using cached tokenizers-0.13.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.5 MB/s eta 0:00:00


In [ ]:
import torch
from convlab2.nlu import NLU
from transformers import BertTokenizerFast, BertForSequenceClassification, BertForTokenClassification
from sklearn.preprocessing import LabelEncoder
import pickle

/content/ConvLab-2/convlab2/dialog_agent/agent.py:86: SyntaxWarning: "is" with a literal. Did you mean "=="?
  self.opponent_name = 'user' if self.name is 'sys' else 'sys'
/content/ConvLab-2/convlab2/dialog_agent/agent.py:128: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self.name is 'sys':
/content/ConvLab-2/convlab2/dialog_agent/agent.py:146: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self.name is 'sys':


In [ ]:
class CustomBERTNLU(NLU):
    def __init__(self, intent_model_path, slot_model_path, tokenizer_path, intent_label_encoder_path, slot_label_encoder_path):
        self.intent_model = BertForSequenceClassification.from_pretrained(intent_model_path)
        self.slot_model = BertForTokenClassification.from_pretrained(slot_model_path)
        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_path)
        
        with open(intent_label_encoder_path, 'rb') as file:
            self.intent_label_encoder = pickle.load(file)
            
        with open(slot_label_encoder_path, 'rb') as file:
            self.slot_label_encoder = pickle.load(file)
            
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.intent_model.to(self.device)
        self.slot_model.to(self.device)
        
    def parse(self, utterance):
        # Intent detection
        input_tokens = self.tokenizer(utterance, return_tensors='pt', truncation=True, padding=True)
        input_tokens = {k: v.to(self.device) for k, v in input_tokens.items()}
        intent_logits = self.intent_model(**input_tokens).logits
        intent_id = torch.argmax(intent_logits, dim=-1).item()
        intent = self.intent_label_encoder.inverse_transform([intent_id])[0]
        
        # Slot filling
        slot_input = self.tokenizer(utterance, return_tensors='pt', return_offsets_mapping=True, truncation=True, padding=True)
        slot_input = {k: v.to(self.device) for k, v in slot_input.items()}
        slot_logits = self.slot_model(**slot_input).logits
        slot_ids = torch.argmax(slot_logits, dim=-1).squeeze(0)
        slot_predictions = self.slot_label_encoder.inverse_transform(slot_ids.cpu())
        slots = []
        
        for prediction, offset_mapping in zip(slot_predictions, slot_input['offset_mapping'][0]):
            if prediction.startswith('B-'):
                slot_name = prediction[2:]
                slot_value = utterance[offset_mapping[0]: offset_mapping[1]]
                slots.append((slot_name, slot_value))
        
        return intent, slots

In [ ]:
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from convlab2.dialog_agent import PipelineAgent
from convlab2.dst.rule.multiwoz import RuleDST
from convlab2.policy.rule.multiwoz import RulePolicy
from convlab2.nlg.template.multiwoz import TemplateNLG


/usr/local/lib/python3.9/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
intent_model_path = "/content/drive/MyDrive/KVRET/trained_kvret_intent_model/"
tokenizer_path = "/content/drive/MyDrive/KVRET/trained_kvret_intent_model/"
intent_label_encoder_path = "/content/drive/MyDrive/KVRET/trained_kvret_intent_model/label_encoder.pkl"
slot_model_path = "/content/drive/MyDrive/KVRET/trained_kvret_slot_model/"
# The tokenizer is the same for both models, so you can use the same path
slot_label_encoder_path = "/content/drive/MyDrive/KVRET/trained_kvret_slot_model/label_encoder.pkl"


In [ ]:
# from transformers import BertConfig, BertTokenizer, BertForSequenceClassification

# config = BertConfig.from_json_file("/content/drive/MyDrive/KVRET/trained_kvret_intent_model/config.json")


In [ ]:
# Create an instance of your custom NLU component
nlu = CustomBERTNLU(
    intent_model_path, 
    slot_model_path, 
    tokenizer_path, 
    intent_label_encoder_path, 
    slot_label_encoder_path
)

In [ ]:
# Set up the dialogue system components
dst = RuleDST()
policy = RulePolicy()
nlg = TemplateNLG(is_user=False)

In [ ]:
agent = PipelineAgent(nlu, dst, policy, nlg, name='sys')

In [ ]:
# Now you can use the agent to process user inputs
user_input = "Can you make a system please?"
agent_response = agent.response(user_input)
print(agent_response)

I ' m happy to have been able to help you today .


In [ ]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge


In [ ]:
# Load the dataset
with open("/content/drive/MyDrive/KVRET/dialogues.json", "r") as f:
    data = json.load(f)

In [ ]:
def process_dialogue(dialogue):
    user_utterances = []
    system_utterances = []
    ground_truth_responses = []

    for turn in dialogue["turns"]:
        if turn["speaker"] == "user":
            user_utterances.append(turn["utterance"])
        elif turn["speaker"] == "system":
            ground_truth_responses.append(turn["utterance"])

    return user_utterances, ground_truth_responses

In [ ]:
bleu_scores = []
# f1_scores = []
rouge_scores = []
rouge = Rouge()
smoothing_function = SmoothingFunction()

In [ ]:
for dialogue in data:
    user_utterances, ground_truth_responses = process_dialogue(dialogue)
    predicted_responses = []

    for user_utterance in user_utterances:
        agent_response = agent.response(user_utterance)
        predicted_responses.append(agent_response)

    # Compute BLEU scores
    for pred, gt in zip(predicted_responses, ground_truth_responses):
        bleu_score = sentence_bleu([gt.split()], pred.split(), weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing_function.method1)
        bleu_scores.append(bleu_score)

    # Compute ROUGE scores
    for pred, gt in zip(predicted_responses, ground_truth_responses):
        if len(pred) == 0 or len(gt) == 0:
            continue
        rouge_score = rouge.get_scores(pred, gt, avg=True)
        rouge_scores.append(rouge_score)


In [ ]:
# Compute ROUGE scores
for pred, gt in zip(predicted_responses, ground_truth_responses):
    if len(pred) == 0 or len(gt) == 0:
        continue
    rouge_score = rouge.get_scores(pred, gt, avg=True)
    rouge_scores.append(rouge_score)


In [ ]:
average_bleu_score = sum(bleu_scores) / len(bleu_scores)
average_rouge_scores = {
    metric: sum(scores[metric]["f"] for scores in rouge_scores) / len(rouge_scores) for metric in ["rouge-1", "rouge-2", "rouge-l"]
}

print("Average BLEU score:", average_bleu_score)
print("Average ROUGE scores:", average_rouge_scores)

Average BLEU score: 0.005800435430076993
Average ROUGE scores: {'rouge-1': 0.05088082779550414, 'rouge-2': 0.004514755442310943, 'rouge-l': 0.04869316174412618}


In [ ]:
def chat():
    print("Welcome to the Interactive Dialogue System!")
    print("You can start chatting with the agent by typing a message below.")
    print("To exit the chat, type 'quit'.\n")
    
    while True:
        user_input = input("User: ")
        
        if user_input.lower() == 'quit':
            print("Goodbye!")
            break
        
        # Generate agent's response using your trained model
        agent_response = generate_response(user_input)  # Replace this function with your model's response generation method
        
        print("Agent:", agent_response)

In [ ]:
def generate_response(user_input):
    # Implement your model's response generation logic here
    response =agent.response(user_input)
    return response

In [ ]:
chat()

Welcome to the Interactive Dialogue System!
You can start chatting with the agent by typing a message below.
To exit the chat, type 'quit'.

User: hello
Agent: Thank you for using our services .
User: I want to schedule time for 7pm
Agent: Thank you for using our services .
User: how are you
Agent: Glad to have been of help . Thank you for using the Cambridge TownInfo centre . Enjoy the rest of your day !.
User: why is this system bad
Agent: Thank you for using our services .


KeyboardInterrupt: ignored